# Padronização dos dados com o Portal Brasileiro de Dados Abertos (CGU)
### Limpeza da lista de instituições e construção da correspondência entre temas da CGU e da Câmara dos Deputados

Queremos padronizar o que for possível dos nossos dados com os dados daqui: <https://dados.gov.br/dados/reusos>. 

In [1]:
import json

## Functions

In [2]:
def load_json(filename):
    
    with open(filename, 'r') as f:
        data = json.load(f)
    
    return data

## ETL institution data

In [77]:
from glob import glob
from collections import defaultdict

In [85]:
# Load institution names:
inst_files = glob('../dados/aux/organizacao_*.json')
institutions = []
ids = []
for f in inst_files:
    institutions += [(i['titulo'], i['id']) for i in load_json(f)]
# Sort list:
institutions = sorted(institutions)
# Create translator (name > ID):
inst2id = defaultdict(lambda: None, dict(institutions))

In [88]:
institutions

[('Advocacia-Geral da União - AGU', '96791cba-af6a-4f90-9ac4-7c74e1ab9ee4'),
 ('Agência Espacial Brasileira - AEB', '637b53d0-a807-4742-8ab7-3813f0f860a9'),
 ('Agência Nacional de Aviação Civil - ANAC',
  'd5bc559a-beda-4f1e-92ea-380bd5ef12a2'),
 ('Agência Nacional de Energia Elétrica - ANEEL',
  '729e2446-e5a9-4ecf-a2d4-ac08235f55fc'),
 ('Agência Nacional de Mineração', 'dc10a958-f3a0-4afa-acb4-adea38437f7f'),
 ('Agência Nacional de Saúde Suplementar',
  '3d6aab5a-3179-4c4a-b84a-59e0d54d1c1a'),
 ('Agência Nacional de Telecomunicações',
  '5fbc22fd-5842-4bde-b48e-669c4ec95317'),
 ('Agência Nacional de Transportes Aquaviários',
  '37b91e7c-2d92-43a8-b26f-f5df18c0a903'),
 ('Agência Nacional de Transportes Terrestres',
  '7e70b268-dd67-4cbd-bfba-c27206f1602b'),
 ('Agência Nacional de Vigilância Sanitária - ANVISA',
  '1a04b59b-c243-40d8-95f8-5cf3e10e4aea'),
 ('Agência Nacional de Águas e Saneamento Básico',
  '39868dc2-d6b8-4b89-84e4-692c2fe17b25'),
 ('Agência Nacional do Cinema', '89819e

### Checks

In [40]:
len(institutions) == len(set(institutions))

True

## ETL topic data

In [8]:
# CGU topics:
cgu_topics_raw = load_json('../dados/aux/temas_cgu_v1.json')
cgu_topics = [c['title'] for c in cgu_topics_raw]

In [8]:
# CORDATA v01 topics:
with open('../dados/aux/temas_cordata_v1.json', 'r') as f:
    cordata_topics = [line.rstrip() for line in f]

In [15]:
# Manually created translation dict:
cordata2cgu = {'Agropecuária':'Agricultura, extrativismo e pesca',
               'Administração Pública':'Governo e Política',
               'Alimentação':'Assistência e Desenvolvimento Social',
               'Arte e Cultura':'Cultura, Lazer e Esporte',
               'Atividade industrial, de comércio ou serviços':'Comércio, Serviços e Turismo',
               'Ciência e Tecnologia':'Ciência, Informação e Comunicação',
               'Comunicações':'Ciência, Informação e Comunicação',
               'Consumo':'Economia e Finanças',
               'Dados demográficos':'Dados Estratégicos',
               'Defesa':'Defesa e Segurança',
               'Direito e Processual Penal':'Justiça e Legislação',
               'Direitos Humanos':'Assistência e Desenvolvimento Social',
               'Economia':'Economia e Finanças',
               'Esporte':'Cultura, Lazer e Esporte',
               'Finanças e Orçamento Público':'Economia e Finanças',
               'Imóveis, Habitação e Urbanismo':'Habitação, Saneamento e Urbanismo',
               'Justiça e Direito':'Justiça e Legislação',
               'Lazer':'Cultura, Lazer e Esporte',
               'Multimídia':'Cultura, Lazer e Esporte',
               'Política, Partidos e Eleições':'Governo e Política',
               'Política, Partidos e Eleições': 'Governo e Política',
                'Previdência e Assistência Social': 'Assistência e Desenvolvimento Social',
                'Processo Legislativo': 'Governo e Política',
                'Recursos hídricos': 'Meio Ambiente',
                'Recursos minerais': 'Economia e Finanças',
                'Redes Sociais': 'Ciência, Informação e Comunicação',
                'Religião': 'Cultura, Lazer e Esporte',
                'Segurança': 'Defesa e Segurança',
                'Terras': 'Geografia',
                'Trabalho e Emprego': 'Trabalho',
                'Transparência': 'Governo e Política',
                'Transporte': 'Transportes e Trânsito',
                'Turismo': 'Comércio, Serviços e Turismo'
               }

In [89]:
len(cgu_topics)

22

In [45]:
for t in cgu_topics:
    print(t)

Agricultura, extrativismo e pesca
Assistência e Desenvolvimento Social
Ciência, Informação e Comunicação
Comércio, Serviços e Turismo
Cultura, Lazer e Esporte
Dados Estratégicos
Defesa e Segurança
Economia e Finanças
Educação
Energia
Equipamentos Públicos
Geografia
Governo e Política
Habitação, Saneamento e Urbanismo
Indústria
Justiça e Legislação
Meio Ambiente
Plano Plurianual
Relações Internacionais
Saúde
Trabalho
Transportes e Trânsito


### List all topics

In [53]:
import pandas as pd

import clean as cc

In [54]:
# Load translations:
translation_df = pd.read_excel('../docs/conteudo_site/textos_site_traducao_traduzidos_v04.xlsx', sheet_name='Site')

# Build translators:
tema_map = cc.get_translator(translation_df, r'filtro_tema_\d')

In [59]:
def wrap(topics):
    usecase = {'topics': topics}
    data = [usecase]
    meta = {'data':data}
    return meta

In [60]:
# Save to file:
if True:
    with open('/home/hxavier/temp/cgu_topics.json', 'w') as f:
        json.dump(wrap(cgu_topics), f, ensure_ascii=False, indent=2)

In [61]:
es = [tema_map[t] for t in cgu_topics]

In [62]:
# Save to file:
if True:
    with open('/home/hxavier/temp/cgu_topics_es.json', 'w') as f:
        json.dump(wrap(es), f, ensure_ascii=False, indent=2)

In [64]:
# Check if JSON is correct after manual modification:
check_ok = load_json('../dados/limpos/usecases_n12_2024-01-02.json')

### Checks

In [19]:
set(cordata_topics) - set(cordata2cgu.keys())

{'Educação', 'Energia', 'Meio Ambiente', 'Relações Internacionais', 'Saúde'}

In [24]:
set(cgu_topics) - set(cordata2cgu.values())

{'Educação',
 'Energia',
 'Equipamentos Públicos',
 'Indústria',
 'Plano Plurianual',
 'Relações Internacionais',
 'Saúde'}

In [23]:
set(cordata2cgu.values()) - set(cgu_topics)

set()

In [27]:
extra_trad = sorted(list(set(cgu_topics) - set(cordata_topics)))

### Exports

In [31]:
# For exporting to Excel for translation:
for t in extra_trad:
    print(t)

Agricultura, extrativismo e pesca
Assistência e Desenvolvimento Social
Ciência, Informação e Comunicação
Comércio, Serviços e Turismo
Cultura, Lazer e Esporte
Dados Estratégicos
Defesa e Segurança
Economia e Finanças
Equipamentos Públicos
Geografia
Governo e Política
Habitação, Saneamento e Urbanismo
Indústria
Justiça e Legislação
Plano Plurianual
Trabalho
Transportes e Trânsito
